In [ ]:
#スクレイピングでデータ抽出（ビットコイン）
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd

chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(executable_path =r'C:\\chromedriver-96\\chromedriver.exe' )
driver.get("https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=20130428&end=")
time.sleep(5)
for i in range(150):
    button = driver.find_element_by_css_selector("#__next > div > div > div.sc-57oli2-0.comDeo.cmc-body-wrapper > div > div.sc-16r8icm-0.jKrmxw.container > div > div > p:nth-child(3) > button")
    driver.execute_script("arguments[0].scrollIntoView(false);", button)
    button.click()
    time.sleep(1)
html = driver.page_source
bitcoin_market_info = pd.read_html(html+time.strftime("%Y%m%d"))[0]
driver.close()

# Dateを文字列から日付フォーマットへ変換
bitcoin_market_info = bitcoin_market_info.assign(Date=pd.to_datetime(bitcoin_market_info['Date']))
 
# 取引高が'-'の欠損データを0へ変換
bitcoin_market_info.loc[bitcoin_market_info['Volume']=="-",'Volume']=0

# イントへ変換
column_list = bitcoin_market_info.columns[1:]
for column in column_list:
    bitcoin_market_info[column] = bitcoin_market_info[column].str.replace(",","")
    bitcoin_market_info[column] = bitcoin_market_info[column].str.replace("$","")
    bitcoin_market_info[column] = bitcoin_market_info[column].astype("float64")
# データセットのヘッド情報の確認
bitcoin_market_info.head()

In [ ]:
# イーサリアムの価格データを所定のURLからスクレイピング
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(executable_path =r'C:\\chromedriver-96\\chromedriver.exe' )
driver.get("https://coinmarketcap.com/currencies/ethereum/historical-data/?start=20130428&end=")
time.sleep(5)
for i in range(100):
    button = driver.find_element_by_css_selector("#__next > div > div > div.sc-57oli2-0.comDeo.cmc-body-wrapper > div > div.sc-16r8icm-0.jKrmxw.container > div > div > p:nth-child(3) > button")
    driver.execute_script("arguments[0].scrollIntoView(false);", button)
    button.click()
    time.sleep(1)
html = driver.page_source
eth_market_info = pd.read_html(html+time.strftime("%Y%m%d"))[0]
driver.close()
# Dateを文字列から日付フォーマットへ変換
eth_market_info = eth_market_info.assign(Date=pd.to_datetime(eth_market_info['Date']))
 
# 取引高が'-'の欠損データを0へ変換
eth_market_info.loc[eth_market_info['Volume']=="-",'Volume']=0
 
# イントへ変換
for column in column_list:
    eth_market_info[column] = eth_market_info[column].str.replace(",","")
    eth_market_info[column] = eth_market_info[column].str.replace("$","")
    eth_market_info[column] = eth_market_info[column].astype("float64")
# データセットのヘッド情報の確認
eth_market_info.head()

In [ ]:
# ビットコインとイーサリアムのデータフレームのカラム名を変更
bitcoin_market_info.columns =[bitcoin_market_info.columns[0]]+['bt_'+i for i in bitcoin_market_info.columns[1:]]
eth_market_info.columns =[eth_market_info.columns[0]]+['eth_'+i for i in eth_market_info.columns[1:]]
bitcoin_market_info=bitcoin_market_info.rename(columns={"bt_Open*":"bt_Open"})
bitcoin_market_info=bitcoin_market_info.rename(columns={"bt_Close**":"bt_Close"})
eth_market_info=eth_market_info.rename(columns={"eth_Open*":"eth_Open"})
eth_market_info=eth_market_info.rename(columns={"eth_Close**":"eth_Close"})

In [ ]:
#csv形式に保存
bitcoin_market_info.to_csv("data/bitcoin_market_info.csv")
eth_market_info.to_csv("data/eth_market_info.csv")